# This is a web scrapper for Intenship and Entry Level Jobs In Nigeria

## We will attempt to scrape data from a Popular Web Site in Nigeria(Nairaland). There is a Thread on the website with frequent web listing for Entry level Jobs

In [1]:
#Let's import the required library
import requests
import bs4
req = requests.get("https://www.nairaland.com/3304020/internships-entry-level-jobs-nigeria")

### We will export the content to a textfile to see what we have in it

In [2]:
file = open(r'extract.txt', 'w', encoding="utf-8")
file.write(req.text)
file.close()

In [3]:
fl = open(r'extract.txt').read()
page = bs4.BeautifulSoup(fl)

Our mission for now is to determine the number of pages the thread has. Looking at the textfile we would see that the information can be found in a class 'nocopy' and within the 'href' tag.

In [4]:
elements = page.select('.nocopy a')
elements[0]

<a href="/3304020/internships-entry-level-jobs-nigeria/1">(2)</a>

Now, we have a challenge, There are multiple elements with a class 'nocopy' and a tag of 'href'

In [5]:
#Let's use a filter to remove elements not starting with '<a href="/3304020'. That way we can get the last pageLet
actual = list(filter(lambda x: str(x)[:17] == '<a href="/3304020', elements))
actual[-1]

<a href="/3304020/internships-entry-level-jobs-nigeria/66">(67)</a>

Let's get the number of the last page

In [6]:
def lastPageIndex(st):
    x,y = st.index('('), st.index(')')
    v = st[x+1:y]
    return int(v)
num_page = lastPageIndex(str(actual[-1]))
num_page

67

We can now see that the thread has 67 pages(at the time this project was executed), with the last page been 67-1.
let us scrape the last and second to the last pages.

In [7]:
req1 = requests.get("https://www.nairaland.com/3304020/internships-entry-level-jobs-nigeria/"+str(num_page-1))
req2 = requests.get("https://www.nairaland.com/3304020/internships-entry-level-jobs-nigeria/"+str(num_page-2))
preg = str(req1.text) + str(req2.text)
file = open(r'extract1.txt', 'w', encoding="utf-8")
file.write(preg)
file.close()

After going through the file we can see that the Job lists are in the 'table' tag and in the class 'narrow'

In [8]:
file = open(r'extract1.txt',encoding="utf-8").read()
page = bs4.BeautifulSoup(file)
element = page.select('table .narrow') 

In [9]:
#Let us check the first element
element[0]

<div class="narrow"><a href="https://careersome.com/site/job?id=male-administrative-staff-at-pickmeup-international-company-owerri-imo">https://careersome.com/site/job?id=male-administrative-staff-at-pickmeup-international-company-owerri-imo</a><br/>Male Administrative Staff​ at Pickmeup International Company - Owerri, Imo​.<br/><br/><a href="https://careersome.com/site/job?id=loan-officer-at-workforce-group">https://careersome.com/site/job?id=loan-officer-at-workforce-group</a><br/>Loan Officer at ​Workforce Group<br/><br/><a href="https://careersome.com/site/job?id=business-developer-at-romanspage-global">https://careersome.com/site/job?id=business-developer-at-romanspage-global</a><br/>Business Developer at ​Romanspage Global<br/><br/><a href="https://careersome.com/site/job?id=client-relationship-officer-at-romanspage-global">https://careersome.com/site/job?id=client-relationship-officer-at-romanspage-global</a><br/>Client Relationship Officer at ​Romanspage Global<br/><br/><a href

We can see that each Job Offer and it's link is separated with the 'br' tag

In [10]:
#Let's repalce the <br/> tag with a new line character \n. This will enable us split lines.
first_element = element[0]
first_element = str(first_element).replace('<br/>','\n')
#We need to remove the unwanted charcaters \u200b 
first_element = str(first_element).replace('\u200b','').splitlines()
#some elements would be empty, we need to get rid of them
first_element = list(filter(lambda x: x != '', first_element))
first_element

['<div class="narrow"><a href="https://careersome.com/site/job?id=male-administrative-staff-at-pickmeup-international-company-owerri-imo">https://careersome.com/site/job?id=male-administrative-staff-at-pickmeup-international-company-owerri-imo</a>',
 'Male Administrative Staff at Pickmeup International Company - Owerri, Imo.',
 '<a href="https://careersome.com/site/job?id=loan-officer-at-workforce-group">https://careersome.com/site/job?id=loan-officer-at-workforce-group</a>',
 'Loan Officer at Workforce Group',
 '<a href="https://careersome.com/site/job?id=business-developer-at-romanspage-global">https://careersome.com/site/job?id=business-developer-at-romanspage-global</a>',
 'Business Developer at Romanspage Global',
 '<a href="https://careersome.com/site/job?id=client-relationship-officer-at-romanspage-global">https://careersome.com/site/job?id=client-relationship-officer-at-romanspage-global</a>',
 'Client Relationship Officer at Romanspage Global',
 '<a href="https://careersome.co

### We have been able to extract the first Job list in the first Post

Let's Check each Job listed in the post. Note that the first element in the list contains the link to the Job while the next contains the description.

In [11]:
pg = bs4.BeautifulSoup(first_element[0])
pg.select('a')[0].getText()

'https://careersome.com/site/job?id=male-administrative-staff-at-pickmeup-international-company-owerri-imo'

In [12]:
first_element[1]

'Male Administrative Staff at Pickmeup International Company - Owerri, Imo.'

Let Us create a method that extracts the details from all the post and taking into account that all the post aren't in the required format. We ask well group each Job description with it's link

In [13]:
def extractGroup(ob):
    #We need to ensure the length of the list is even (Job list followed by description)
    h = (len(ob)//2)*2
    smallList = []
    bigList = []
    for i in range(h):
        if i % 2 == 0:
            pg = bs4.BeautifulSoup(ob[i])
            #Take care of elements without the 'a' tag
            try:
                sl = pg.select('a')[0].getText()
                smallList.append(sl)
            except:
                smallList.append('')
        else:
            smallList.append(ob[i])
            y = (ob[i],smallList[i-1])
            bigList.append( y )
    return bigList

We now use the procedure we use to get required content from the first list for all the list

In [14]:
JobList = []
for elem in element:
    elem_replace = str(elem).replace('<br/>','\n').replace('\u200b','').splitlines()
    remove_empty = list(filter(lambda x: x != '', elem_replace))
    new_list = extractGroup(remove_empty)
    JobList.extend(new_list)

In [15]:
#checking the first Job details
JobList[0]

('Male Administrative Staff at Pickmeup International Company - Owerri, Imo.',
 'https://careersome.com/site/job?id=male-administrative-staff-at-pickmeup-international-company-owerri-imo')

## Now let's search our Job List for Jobs

In [16]:
import re

In [17]:
search_query = 'data analyst'
#for each Job offer we check if any word in the search Query is in our Job list
update_query = search_query.lower().split()
new_query = '|'.join(update_query)
comp = re.compile(new_query)
for job in JobList:
    if comp.search(job[0].lower()):
        print(job[0])
        print(job[1])

Citibank Middle East and Africa Analyst Program 2020, Nigeria</div>
https://careersome.com/site/job?id=citibank-middle-east-and-africa-analyst-program-2020-nigeria
Procurement Analyst at Arnergy Solar Limited
https://careersome.com/site/job?id=procurement-analyst-at-arnergy-solar-limited
Pricing &amp; Revenue Management Analyst at Finchglow Travels
https://careersome.com/site/job?id=pricing-revenue-management-analyst-at-finchglow-travels
Pricing Analyst at Paykobo.com
https://careersome.com/site/job?id=pricing-analyst-at-paykobocom
Quality Control Support Analyst at RxAll Inc - Ibadan
https://careersome.com/site/job?id=quality-control-support-analyst-at-rxall-inc-ibadan
SME Analyst at Enugu SME Agency
https://careersome.com/site/job?id=sme-analyst-at-enugu-sme-agency
Tableau Business Analyst / BI Analyst at Vascon Solutions
https://careersome.com/site/job?id=tableau-business-analyst-bi-analyst-at-vascon-solutions
Credit Risk Analyst - Intern at BoroMe Limited</div>
https://careersome.c

In [18]:
search_query = 'python developer'
#for each Job offer we check if any word in the search Query
update_query = search_query.lower().split()
new_query = '|'.join(update_query)
comp = re.compile(new_query)
for job in JobList:
    if comp.search(job[0].lower()):
        print(job[0])
        print(job[1])

Business Developer at Romanspage Global
https://careersome.com/site/job?id=business-developer-at-romanspage-global
Backend Developer at CloudWare Technologies
https://careersome.com/site/job?id=backend-developer-at-cloudware-technologies-2
Angular Developer at 390labs
https://careersome.com/site/job?id=angular-developer-at-390labs
full stack developers (full time / internship) at Anakle
https://careersome.com/site/job?id=full-stack-developers-full-time-internship-at-anakle
React Developer at The Bulb
https://careersome.com/site/job?id=react-developer-at-the-bulb
Node.js Developer at The Bulb
https://careersome.com/site/job?id=nodejs-developer-at-the-bulb
JavaScrip Developer at The Bulb
https://careersome.com/site/job?id=javascrip-developer-at-the-bulb
Python Web Developer at The Bulb
https://careersome.com/site/job?id=python-web-developer-at-the-bulb
iOS developer at The Bulb
https://careersome.com/site/job?id=ios-developer-at-the-bulb
Backend (Python) Developer Intern at Urban Hive Ni

It would be boring to enter the search text in the program. Let's introduce a simple GUI
We will make use of a library called Traits. Note: traits has to be installed (using pip install)

In [19]:
from traits.api import HasTraits, Str, Range, Enum, Int
from traitsui.api import Item, RangeEditor, View, Label

In [20]:
class Job(HasTraits):
    name = Str('search here')

job = Job()

Job_view = View(
    Label("What Job are you applying for?"),
    Item('name'),
    buttons=['OK', 'Cancel'],
    title = 'Internship and Entry level Job Search',
    resizable = True
)
job.configure_traits(view=Job_view)

True

In [21]:
job.name

'data analyst'

In [22]:
def generateResult(qry):
    result = ''
    update_query = qry.lower().split()
    new_query = '|'.join(update_query)
    comp = re.compile(new_query)
    for job in JobList:
        if comp.search(job[0].lower()):
            result += job[0] + ' \n '
            result += job[1] + ' \n '
    return result

In [23]:
generateResult(job.name)

'Citibank Middle East and Africa Analyst Program 2020, Nigeria</div> \n https://careersome.com/site/job?id=citibank-middle-east-and-africa-analyst-program-2020-nigeria \n Procurement Analyst at Arnergy Solar Limited \n https://careersome.com/site/job?id=procurement-analyst-at-arnergy-solar-limited \n Pricing &amp; Revenue Management Analyst at Finchglow Travels \n https://careersome.com/site/job?id=pricing-revenue-management-analyst-at-finchglow-travels \n Pricing Analyst at Paykobo.com \n https://careersome.com/site/job?id=pricing-analyst-at-paykobocom \n Quality Control Support Analyst at RxAll Inc - Ibadan \n https://careersome.com/site/job?id=quality-control-support-analyst-at-rxall-inc-ibadan \n SME Analyst at Enugu SME Agency \n https://careersome.com/site/job?id=sme-analyst-at-enugu-sme-agency \n Tableau Business Analyst / BI Analyst at Vascon Solutions \n https://careersome.com/site/job?id=tableau-business-analyst-bi-analyst-at-vascon-solutions \n Credit Risk Analyst - Intern a

In [24]:
class Display(HasTraits):
    name = Str('search here')

display = Display()

Display_view = View(
    Label(generateResult(job.name)),
    buttons=['OK', 'Cancel'],
    title = 'Internship and Entry level Job Search',
    resizable = True
)
display.configure_traits(view=Display_view)

True

In [28]:
#Now Run this cell to Enter your Search and display the result in a GUI
if job.configure_traits(view=Job_view):
    display = Display()

    Display_view = View(
    Label(generateResult(job.name)),
    buttons=['OK', 'Cancel'],
    title = 'Internship and Entry level Job Search',
    resizable = True
    )
    display.configure_traits(view=Display_view)

In [26]:
job.name

'data analyst'

# Thank You